# Function Calling

**Function calling** é, de fato, uma estratégia de fornecer ao LLM um “esqueleto” estruturado (normalmente em JSON Schema) descrevendo:

- Nome da função 
- O que ela faz 
- Quais argumentos aceita
- Tipo/formato desses argumentos
- Quais campos são obrigatórios

Isso guia o modelo a:
- Entender quando usar a função
- Como montar corretamente os argumentos
- Devolver uma saída estruturada e válida, reduzindo erros

Ou seja: não é para o LLM executar a função, e sim para pedir corretamente que ela seja executada pelo seu código.

In [32]:
import os, json
from dotenv import dotenv_values
import requests
from openai import OpenAI

config = dotenv_values("../.env")
openai_api_key = config["OPENAI_API_KEY"]
open_weather_key = config["OPEN_WEATHER"]

## Testando API Open Weather

In [33]:
lat = -4.35795
lon = -39.3024
url = (
    f"https://api.openweathermap.org/data/3.0/onecall"
    f"?lat={lat}&lon={lon}"
    f"&appid={open_weather_key}"
    f"&units=metric&lang=pt_br"
) 


resp = requests.get(url)  
resp.status_code

200

In [34]:
result = json.loads(resp.content)
result


{'lat': -4.358,
 'lon': -39.3024,
 'timezone': 'America/Fortaleza',
 'timezone_offset': -10800,
 'current': {'dt': 1769184205,
  'sunrise': 1769157550,
  'sunset': 1769201914,
  'temp': 33.35,
  'feels_like': 33.3,
  'pressure': 1008,
  'humidity': 35,
  'dew_point': 15.81,
  'uvi': 11.41,
  'clouds': 63,
  'visibility': 10000,
  'wind_speed': 4,
  'wind_deg': 115,
  'wind_gust': 4.7,
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'nublado',
    'icon': '04d'}]},
 'minutely': [{'dt': 1769184240, 'precipitation': 0},
  {'dt': 1769184300, 'precipitation': 0},
  {'dt': 1769184360, 'precipitation': 0},
  {'dt': 1769184420, 'precipitation': 0},
  {'dt': 1769184480, 'precipitation': 0},
  {'dt': 1769184540, 'precipitation': 0},
  {'dt': 1769184600, 'precipitation': 0},
  {'dt': 1769184660, 'precipitation': 0},
  {'dt': 1769184720, 'precipitation': 0},
  {'dt': 1769184780, 'precipitation': 0},
  {'dt': 1769184840, 'precipitation': 0},
  {'dt': 1769184900, 'precipitation': 

In [35]:
result["current"]["weather"][0]["description"]

'nublado'

## Functions que serão utilizadas pelo LLM

In [36]:
def get_weather(lat:float, lon:float, api_key=open_weather_key) -> str:

    """ 
    Essa função nos informa o clima  do ponto dado por 
    latitude=lat e longitude=lon no momento exato em que é chamada
    """
    print("chamando a função get_weather")
    
    if not api_key:
        return {"error": "OpenWeather api key not set"}
    
    url = (
    f"https://api.openweathermap.org/data/3.0/onecall"
    f"?lat={lat}&lon={lon}"
    f"&appid={open_weather_key}"
    f"&units=metric&lang=pt_br"
    )

    response = requests.get(url)
    result = json.loads(response.content)

    return f"O clima em lat={lat} e lon={lon}, região da {result["timezone"]}, é {result["current"]["weather"][0]["description"]}"

print(get_weather(8.36666, 115.15))


chamando a função get_weather
O clima em lat=8.36666 e lon=115.15, região da Asia/Manila, é nublado


In [37]:
def get_horoscope(signo:str) -> str:
    """ Essa função retorna o horóscopo de hoje para um determinado signo astrológico """
    return f"{signo}: Próxima sexta você viajará para São Paulo"

print(get_horoscope("libra"))


libra: Próxima sexta você viajará para São Paulo


## Schema para o modelo (Function Calling)

In [41]:
functions = [
    {   
        "type": "function",
        "name": "get_weather",
        "description": "Retorna temperatura e descrição do tempo para coordenadas (lat, lon).",
        "parameters": {
            "type": "object",
            "properties": {
                "lat": {"type": "number", "minimum": -90, "maximum": 90, "description": "Latitude"},
                "lon": {"type": "number", "minimum": -180, "maximum": 180, "description": "Longitude"},
            },
            "required": ["lat", "lon"]
        }
    },
    {   
        "type": "function",
        "name": "get_horoscope",
        "description": "Retorna o horóscopo de hoje para um signo astrológico (ex: Peixes).",
        "parameters": {
            "type": "object",
            "properties": {
                "signo": {"type": "string", "description": "Um signo astrológico como Touro ou Aquário"}
            },
            "required": ["signo"]
        }
    }
]


## Workflow

### 1ª chamada → decide tool

In [42]:
input_list = [{
    "role": "user",
    "content": "Qual o meu horóscopo de hoje? Sou de peixes"
}]

client_openai = OpenAI(api_key=openai_api_key)

response = client_openai.responses.create(
    model="gpt-4.1-mini",
    input=input_list,
    temperature=0.0, 
    tools=functions
)

tool_call = response.output[0]
tool_call


ResponseFunctionToolCall(arguments='{"signo":"Peixes"}', call_id='call_8LMBcBBNncvvXS4XTT6hFtMx', name='get_horoscope', type='function_call', id='fc_0c1855acd3894b3b0069739c4e00888193a4531018b94cbc75', status='completed')

In [43]:
argumento = tool_call.arguments
argumento

'{"signo":"Peixes"}'

In [44]:
arg = json.loads(argumento)
arg["signo"]

'Peixes'

In [45]:
tool_name = tool_call.name
tool_name

'get_horoscope'

In [46]:
if tool_name == "get_horoscope":
    tool_result = get_horoscope(arg["signo"])


### 2ª chamada -> LLM gera resposta: 

In [51]:

# Recria o contexto: pergunta original + resultado da tool (como mensagem "user")
followup_input = [
    {"role": "user", "content": "Qual o meu horóscopo de hoje? Sou de peixes"},
    {
        "role": "user",
        "content": f"[{tool_name} result]\n{tool_result}"
    }
]

In [52]:
followup = client_openai.responses.create(
        model="gpt-4.1-mini",
        input=followup_input,
        temperature=0.0
    )

followup

Response(id='resp_0e128d13db530edb0069739cdaae6c819091078c8f89ed38bd', created_at=1769184474.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_0e128d13db530edb0069739cdb01dc8190aa83ac79b75434d8', content=[ResponseOutputText(annotations=[], text='Seu horóscopo de hoje para Peixes indica que na próxima sexta-feira você terá uma viagem para São Paulo. Pode ser um momento interessante para novas experiências e oportunidades. Aproveite para se preparar e ficar aberto a novidades! Se quiser, posso ajudar com dicas para a viagem ou algo mais.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[], top_p=1.0, background=False, completed_at=1769184475.0, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=Non

In [53]:
followup.output[0].content[0].text

'Seu horóscopo de hoje para Peixes indica que na próxima sexta-feira você terá uma viagem para São Paulo. Pode ser um momento interessante para novas experiências e oportunidades. Aproveite para se preparar e ficar aberto a novidades! Se quiser, posso ajudar com dicas para a viagem ou algo mais.'

## Muito manual né, vamos tentar automatizar isso 

**Observação: O exemplo abaixo não roda em produção**

In [61]:
def run_with_tools_simple(content:str ,tools=functions, model="gpt-4.1-mini"):
    client = OpenAI(api_key=openai_api_key)

    input_list = [{
    "role": "user",
    "content": content
    }]
    
    response = client.responses.create(
        model=model,
        input=input_list,
        tools=tools,
        temperature=0
    )

    while response.output[0].type == "function_call":
        tool_call = response.output[0]
        args = json.loads(tool_call.arguments)

        if tool_call.name == "get_horoscope":
            result = get_horoscope(**args)
        elif tool_call.name == "get_weather":
            result = get_weather(**args)
        else:
            raise ValueError("Tool desconhecida")

        input_list.append({
            "role": "user",
            "content": f"[{tool_call.name} result]\n{result}"
        })

        response = client.responses.create(
            model=model,
            input=input_list,
            temperature=0
        )

    return response.output[0].content[0].text


In [62]:
run_with_tools_simple("Qual o clima de hoje em Botswana?")

chamando a função get_weather


'O clima de hoje em Botswana, na região de Gaborone, está com nuvens dispersas.'